In [1]:
# install fastparquet via system call
# !pip3 install fastparquet
# !pip3 install statsmodels 
# !pip3 install matplotlib

import fastparquet 
import pandas as pd, os
import numpy as np

In [2]:
dir = 'data/'
df = pd.read_parquet(dir + 'hw2_mfin7037_data.parquet')
# start the data in 1994, before which there's some weird stuff
df = df.query("date>='1994-01-01'")
# remember to fitler the data
df.head()

,permno,date,ret,intraday_ret_month,overnight_ret_month,mcap_lag1,prc_lag1,mom_intraday,mom,mom_overnight,mcap_bin
148,10094.0,1994-01-31,0.181818,-0.234356,0.543560,42515.000,5.500,-0.307430,-0.087011,0.229986,4.0
149,10094.0,1994-02-28,-0.115385,0.222005,-0.276094,50245.000,6.500,-0.923227,-0.646627,0.286166,5.0
150,10094.0,1994-03-31,-0.152174,0.187957,-0.286318,44447.500,5.750,-1.428256,-0.712195,0.725624,4.0
151,10094.0,1994-04-29,-0.076923,0.236076,-0.253220,37756.875,4.875,-1.398879,-0.834798,0.564079,4.0
152,10094.0,1994-05-31,0.027778,-0.138717,0.193312,34852.500,4.500,-0.764079,-0.640503,0.123572,4.0


In [3]:
# slicing parts of data
df_head = df.head()

In [4]:
def produce_table(input_data, subsetting=lambda df: df):
    """
    Combines portfolio data from 'bins' and 'pnl', computes summary statistics
    (mean returns multiplied by 100 and one-sample t-test statistics) for each portfolio,
    and returns a pivoted table for portfolios 1..10 and a long-short portfolio.
    
    Parameters:
        input_data (dict): Dictionary with keys:
            - 'bins': a DataFrame with columns ['date','bin','ew','vw',
                      'ew_intraday','vw_intraday','ew_overnight','vw_overnight']
            - 'pnl': a DataFrame with columns ['date','ew','vw',
                      'ew_intraday','vw_intraday','ew_overnight','vw_overnight']
                      (this will be used to compute the long-short portfolio)
        subsetting (function): A function to subset/modify the combined DataFrame.
                               Defaults to the identity function.
    
    Returns:
        pd.DataFrame: A table where each row corresponds to a return measure (e.g., EW, VW, etc.)
                      with the first row showing the mean (×100, rounded to 3 decimals) and the second
                      row showing the t-statistic (in parentheses). The long-short portfolio is labeled "10-1".
    """
    # Extract the required columns from each DataFrame.
    bins_df = input_data['bins'][['date', 'bin', 'ew', 'vw', 
                                  'ew_intraday', 'vw_intraday', 
                                  'ew_overnight', 'vw_overnight']].copy()
    pnl_df = input_data['pnl'][['date', 'ew', 'vw', 
                                'ew_intraday', 'vw_intraday', 
                                'ew_overnight', 'vw_overnight']].copy()
    # Designate the pnl data as portfolio "11" (which later will be renamed to "10-1")
    pnl_df['bin'] = 11

    # Combine the two datasets and apply any subsetting
    combined = pd.concat([bins_df, pnl_df], ignore_index=True)
    combined = subsetting(combined)

    # List of return measure columns
    cols = ['ew', 'vw', 'ew_intraday', 'vw_intraday', 'ew_overnight', 'vw_overnight']

    # Function to compute mean multiplied by 100 and rounded to 3 decimals
    def meanna(x):
        return round(x.mean() * 100, 3)

    # Compute group-wise means by portfolio (bin)
    s1 = combined.groupby('bin')[cols].agg(meanna).reset_index()
    s1_melt = s1.melt(id_vars='bin', var_name='variable', value_name='value')
    s1_pivot = s1_melt.pivot(index='variable', columns='bin', values='value').reset_index()
    s1_pivot['order'] = 1

    # Function to compute one-sample t-test statistic (null: mean=0), rounded to 3 decimals and wrapped in parentheses.
    def ttesting(x):
        x = x.dropna()
        if len(x) == 0:
            return None
        stat, _ = ttest_1samp(x, popmean=0)
        return f"({round(stat, 3)})"

    # Compute group-wise t-statistics by portfolio (bin)
    s2 = combined.groupby('bin')[cols].agg(ttesting).reset_index()
    s2_melt = s2.melt(id_vars='bin', var_name='variable', value_name='value')
    s2_pivot = s2_melt.pivot(index='variable', columns='bin', values='value').reset_index()
    s2_pivot['order'] = 2

    # Combine the mean and t-statistic tables
    table = pd.concat([s1_pivot, s2_pivot], ignore_index=True)
    table = table.sort_values(by=['variable', 'order'])
    # For rows with t-statistics (order 2), clear the 'variable' name
    table.loc[table['order'] == 2, 'variable'] = ''
    table = table.drop(columns=['order'])

    # Format the variable names: uppercase and replace underscores with spaces
    table['variable'] = table['variable'].str.upper().str.replace('_', ' ', regex=False)
    table = table.rename(columns={'variable': 'Portfolio'})

    # Rename portfolio 11 to "10-1" (i.e. the long-short portfolio)
    if 11 in table.columns:
        table = table.rename(columns={11: '10-1'})
    
    # Ensure that only portfolios 1 through 10 and the long-short ("10-1") remain.
    valid_portfolios = list(range(1, 11)) + ['10-1']
    cols_to_keep = ['Portfolio'] + [col for col in table.columns if col in valid_portfolios]
    table = table[cols_to_keep]

    return table


# --- Helper: Apply quantiles within each date group ---
def apply_quantiles(df, col, bins=10):
    """
    Assigns a quantile-based bin (1,...,bins) for each value in `col`
    within each date group.
    """
    def quantile_bin(s):
        # Use pd.qcut to get quantile bins; if duplicate edges occur, use rank instead.
        try:
            return pd.qcut(s, q=bins, labels=False, duplicates="drop") + 1
        except Exception:
            return np.ceil(s.rank(method='average') / len(s) * bins)
    return df.groupby('date')[col].transform(quantile_bin)




In [5]:

import matplotlib.pyplot as plt

from scipy.stats import ttest_1samp
import statsmodels.formula.api as smf


def routine_pnl(dta, plot=False):
    """
    Computes portfolio returns (EW/VW) by bin, optionally creates two plots 
    (mean returns by bin and cumulative pnl), runs a series of OLS regressions, 
    and returns a dictionary with outputs.
    
    Assumes dta contains columns: date, bin, ret, mcap_lag1, 
    intraday_ret_month, overnight_ret_month.
    
    Parameters:
      dta   : pd.DataFrame
              Input data.
      plot  : bool, default False
              If True, plots are created; if False, they are suppressed.
    
    Returns:
      dict with keys:
          'p1'             : Axes for decile plot (or None if plot==False)
          'pnl_curve'      : Axes for cumulative pnl plot (or None if plot==False)
          'factor_loadings': Dictionary of regression results
          'pnl'            : DataFrame of pnl calculations
          'bins'           : DataFrame of computed bin returns
    """
    # Ensure date is datetime
    dta = dta.copy()
    dta['date'] = pd.to_datetime(dta['date'])
    
    # Helper: Weighted mean
    def weighted_mean(x, w):
        return np.average(x, weights=w)
    
    # Compute portfolio returns by date and bin
    bins = (dta.groupby(['date', 'bin'])
            .apply(lambda g: pd.Series({
                'ew': g['ret'].mean(),
                'vw': weighted_mean(g['ret'], g['mcap_lag1']),
                'ew_intraday': g['intraday_ret_month'].mean() if 'intraday_ret_month' in g.columns else np.nan,
                'vw_intraday': weighted_mean(g['intraday_ret_month'], g['mcap_lag1']) if 'intraday_ret_month' in g.columns else np.nan,
                'ew_overnight': g['overnight_ret_month'].mean() if 'overnight_ret_month' in g.columns else np.nan,
                'vw_overnight': weighted_mean(g['overnight_ret_month'], g['mcap_lag1']) if 'overnight_ret_month' in g.columns else np.nan,
            }))
            .reset_index())
    
    # Create decile plot (collapsed across dates) if plotting is requested
    if plot:
        bins_collapsed = bins.groupby('bin')[['ew','vw']].mean().reset_index()
        fig1, ax1 = plt.subplots()
        ax1.plot(bins_collapsed['bin'], bins_collapsed['ew'], label='EW', marker='')
        ax1.plot(bins_collapsed['bin'], bins_collapsed['vw'], label='VW', marker='')
        ax1.set_xlabel('Bin')
        ax1.set_ylabel('Return')
        ax1.set_title('Mean Portfolio Returns by Bin')
        ax1.legend()
        plt.tight_layout()
    else:
        ax1 = None
    
    # Compute long-short pnl using the extreme bins (assume bin 1 and max(bin))
    max_bin = bins['bin'].max()
    selected = bins[bins['bin'].isin([1, max_bin])].copy()
    selected['weight'] = selected['bin'].apply(lambda x: -1 if x == 1 else 1)
    pnl = (selected.groupby('date')
           .apply(lambda g: pd.Series({
               'ew': (g['weight'] * g['ew']).sum(),
               'vw': (g['weight'] * g['vw']).sum(),
               'ew_intraday': (g['weight'] * g['ew_intraday']).sum() if 'ew_intraday' in g.columns else np.nan,
               'vw_intraday': (g['weight'] * g['vw_intraday']).sum() if 'vw_intraday' in g.columns else np.nan,
               'ew_overnight': (g['weight'] * g['ew_overnight']).sum() if 'ew_overnight' in g.columns else np.nan,
               'vw_overnight': (g['weight'] * g['vw_overnight']).sum() if 'vw_overnight' in g.columns else np.nan,
           })).reset_index())
    pnl = pnl.sort_values('date')
    pnl['cumvw'] = (1 + pnl['vw']).cumprod() - 1
    pnl['cumew'] = (1 + pnl['ew']).cumprod() - 1
    
    # Create cumulative pnl plot if plotting is requested
    if plot:
        fig2, ax2 = plt.subplots()
        ax2.plot(pnl['date'], pnl['cumew'], label='EW')
        ax2.plot(pnl['date'], pnl['cumvw'], label='VW')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Cumulative Return')
        ax2.set_title('Cumulative PnL Over Time')
        ax2.legend()
        plt.tight_layout()
    else:
        ax2 = None
    
    # Minimal regressions for demonstration, not done tho
    formulas = [
        'ew ~ 1',
        'vw ~ 1',
        'ew ~ 0',  # dummy formula if needed
    ]
    regs = {}
    for bin_val in sorted(bins['bin'].unique()):
        data_bin = bins[bins['bin'] == bin_val]
        for fml in formulas:
            key = f'bin {bin_val} fml {fml}'
            try:
                model = smf.ols(formula=fml, data=data_bin).fit()
            except Exception as e:
                model = None
                print(f"Regression failed for bin {bin_val} with formula '{fml}': {e}")
            regs[key] = model
            
    # Return outputs in a dictionary
    return {
        'p1': ax1,                # decile plot (or None)
        'pnl_curve': ax2,         # cumulative pnl plot (or None)
        'factor_loadings': regs,  # regression results
        'pnl': pnl,               # pnl table
        'bins': bins              # bins table (with computed returns)
    }


In [6]:

# --- Strategy 1 ---
# (Following the R code, first a ranking step is done then overwritten by quantiles on mom_intraday)
res2_strategy1 = df.copy()
# Here we assign bin using mom_intraday quantiles (10 bins)
res2_strategy1['bin'] = apply_quantiles(res2_strategy1, 'mom_overnight', bins=10)
subset1 = res2_strategy1.dropna(subset=['bin', 'mcap_lag1', 'ret'])
strat1 = routine_pnl(subset1)
table1 = produce_table({'bins': strat1['bins'], 'pnl': strat1['pnl']})
print("=== Strategy 1 Table ===")
# print the table
#  order by index and plot table
table1



Regression failed for bin 1.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 2.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 3.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 4.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 5.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 6.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 7.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bin 8.0 with formula 'ew ~ 0': zero-size array to reduction operation maximum which has no identity
Regression failed for bi

/var/folders/2_/grxkzw0s30v2w9rt2lmcr7y80000gn/T/ipykernel_16671/163406224.py:40: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({
/var/folders/2_/grxkzw0s30v2w9rt2lmcr7y80000gn/T/ipykernel_16671/163406224.py:69: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


bin,Portfolio,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,10-1
0,EW,1.079,0.959,1.014,1.041,1.111,1.018,1.074,1.102,1.144,0.854,-0.225
6,,(2.985),(3.019),(3.483),(3.744),(4.087),(3.776),(3.653),(3.274),(2.764),(1.501),(-0.763)
1,EW INTRADAY,9.917,2.542,1.596,1.12,0.876,0.454,3.825,-0.43,-1.34,-4.928,-14.845
7,,(8.747),(9.154),(6.448),(4.916),(3.944),(2.083),(1.035),(-1.64),(-4.285),(-12.419),(-13.394)
2,EW OVERNIGHT,-2.472,-0.543,0.117,0.388,0.676,1.006,1.501,2.363,3.984,10.973,13.445
8,,(-8.599),(-3.186),(0.789),(2.776),(4.932),(7.389),(9.964),(13.119),(16.585),(21.939),(26.711)
3,VW,0.427,0.876,0.682,0.925,0.76,0.904,0.915,0.754,1.17,0.757,0.33
9,,(1.115),(2.928),(2.621),(3.784),(3.278),(3.94),(3.719),(2.815),(2.988),(1.362),(0.815)
4,VW INTRADAY,3.281,1.719,1.035,0.836,0.429,0.281,-0.111,-0.691,-1.393,-3.712,-5.868
10,,(7.04),(5.688),(4.197),(3.839),(2.178),(1.375),(-0.541),(-2.951),(-4.398),(-7.932),(-13.577)


#### Question 5

In [ ]:
# Create bins for intraday momentum and overnight momentum (5 bins each)
bin_df = df.copy()
bin_df['intraday_bin'] = pd.qcut(bin_df['mom_intraday'], 5, labels=False) + 1
bin_df['overnight_bin'] = pd.qcut(bin_df['mom_overnight'], 5, labels=False) + 1

# Group by year and month
bin_df['year_month'] = pd.to_datetime(bin_df['date']).dt.to_period('M')

# Compute value-weighted and equal-weighted returns for each bin combination (5x5 grid)
def compute_monthly_returns(group):
    # Value-weighted return: weighted by mcap_bin
    value_weighted_return = (group['ret'] * group['mcap_bin']).sum() / group['mcap_bin'].sum()
    
    # Equal-weighted return: average of returns
    equal_weighted_return = group['ret'].mean()
    
    return pd.Series({
        'value_weighted_return': value_weighted_return,
        'equal_weighted_return': equal_weighted_return
    })

# Apply the computation for each combination of intraday and overnight bins
monthly_returns = bin_df.groupby(['year_month', 'intraday_bin', 'overnight_bin']).apply(compute_monthly_returns).reset_index()

# Pivot the data to create the 5x5 grid for both value-weighted and equal-weighted returns
value_weighted_grid = monthly_returns.pivot_table(index='intraday_bin', columns='overnight_bin', values='value_weighted_return', aggfunc='mean')
# Pivot with clear column labels and transpose to get correct orientation
value_weighted_grid = monthly_returns.pivot_table(
    index='overnight_bin', 
    columns='intraday_bin', 
    values='value_weighted_return', 
    aggfunc='mean'
).rename_axis(index='Overnight Momentum Bin', columns='Intraday Momentum Bin')

equal_weighted_grid = monthly_returns.pivot_table(
    index='overnight_bin',
    columns='intraday_bin',
    values='equal_weighted_return',
    aggfunc='mean'
).rename_axis(index='Overnight Momentum Bin', columns='Intraday Momentum Bin')

# Display grids with formatted headers
print("Value-Weighted Return Grid (Rows: Overnight Momentum, Columns: Intraday Momentum)")
print(value_weighted_grid.reset_index().to_string(index=False))

print("\nEqual-Weighted Return Grid (Rows: Overnight Momentum, Columns: Intraday Momentum)")
print(equal_weighted_grid.reset_index().to_string(index=False))


Value-Weighted Return Grid (Rows: Overnight Momentum, Columns: Intraday Momentum)
 Overnight Momentum Bin       1.0      2.0      3.0      4.0      5.0
                    1.0  0.006053 0.003161 0.002842 0.005434 0.007782
                    2.0  0.000752 0.004371 0.007278 0.009180 0.010251
                    3.0 -0.001915 0.007530 0.009792 0.010257 0.013949
                    4.0  0.002464 0.008355 0.010178 0.011908 0.016984
                    5.0  0.001905 0.008193 0.012089 0.011331 0.016390

Equal-Weighted Return Grid (Rows: Overnight Momentum, Columns: Intraday Momentum)
 Overnight Momentum Bin      1.0      2.0      3.0      4.0      5.0
                    1.0 0.013916 0.009495 0.005735 0.006768 0.010105
                    2.0 0.006002 0.006167 0.007688 0.010108 0.011444
                    3.0 0.001745 0.007601 0.010213 0.010776 0.015672
                    4.0 0.003703 0.008641 0.011312 0.012843 0.017597
                    5.0 0.005966 0.009717 0.012640 0.013049 0.016786


/var/folders/2_/grxkzw0s30v2w9rt2lmcr7y80000gn/T/ipykernel_16671/3752150714.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_returns = bin_df.groupby(['year_month', 'intraday_bin', 'overnight_bin']).apply(compute_monthly_returns).reset_index()


In [7]:
apply_quantiles(df.copy(), 'mom_overnight', bins=5)

148        5.0
149        5.0
150        5.0
151        5.0
152        4.0
          ... 
3776102    3.0
3776103    5.0
3776104    5.0
3776105    5.0
3776106    5.0
Name: mom_overnight, Length: 1734650, dtype: float64

In [8]:
apply_quantiles(df.copy(), 'mom_intraday', bins=5)

148        1.0
149        1.0
150        1.0
151        1.0
152        1.0
          ... 
3776102    1.0
3776103    1.0
3776104    1.0
3776105    1.0
3776106    1.0
Name: mom_intraday, Length: 1734650, dtype: float64